In [56]:
import requests
from requests import Response
url = "https://tpnco.blob.core.windows.net/blobfs/Rally/TodayUrgentCase.json"
try:
    response:Response = requests.get(url)
    response_json = response.json()
    data = response.json()
    print(response.text)
except Exception as e:
    print(e)


{"type": "FeatureCollection","features": [
{"type": "Feature",	"geometry":{"type": "Polygon",	"coordinates": [[[308552.16,2773353.307],[308546.447,2773362.631],[308545.636,2773362.174],[308539.956,2773359.086],[308517.369,2773346.862],[308510.022,2773342.859],[308497.119,2773336.425],[308489.019,2773332.423],[308481.476,2773329.017],[308475.489,2773326.319],[308469.678,2773324.052],[308453.346,2773317.682],[308451.226,2773316.857],[308446.581,2773315.169],[308439.543,2773312.984],[308436.822,2773312.272],[308424.699,2773309.131],[308421.539,2773308.3],[308415.129,2773306.616],[308409.505,2773305.287],[308404.667,2773304.15],[308393.057,2773301.279],[308380.613,2773298.3],[308390.234,2773282.476],[308393.066,2773283.682],[308405.821,2773288.708],[308409.715,2773290.256],[308426.728,2773297.257],[308435.25,2773299.903],[308440.96,2773301.685],[308443.217,2773302.485],[308465.978,2773310.553],[308487.85,2773319.679],[308503.874,2773327.261],[308507.349,2773329.014],[308513.711,2773332.268

In [54]:
import requests
from requests import Response
import csv
import pandas as pd
import os
import sqlite3
from pyproj import Proj, transform

In [80]:
'''得到資料裡面的coordinates 第0個欄位是座標屬於的資料'''
result = []

# 遍歷 features 中的每個 JSON 物件
for feature in data["features"]:
    # 提取該 feature 的 geometry["0"] 中的所有座標
    coordinates_list = feature["geometry"]["coordinates"][0]
    
#     # 提取該 feature 的 BILL_CODE
    bill_code = feature["properties"]["BILL_CODE"]
    
    # # 遍歷每個座標，將 BILL_CODE 和座標結合，並加入結果列表
    for coordinates in coordinates_list:
        result.append([bill_code] + coordinates)

# 顯示結果
display(result)

[['10967113574169', 308552.16, 2773353.307],
 ['10967113574169', 308546.447, 2773362.631],
 ['10967113574169', 308545.636, 2773362.174],
 ['10967113574169', 308539.956, 2773359.086],
 ['10967113574169', 308517.369, 2773346.862],
 ['10967113574169', 308510.022, 2773342.859],
 ['10967113574169', 308497.119, 2773336.425],
 ['10967113574169', 308489.019, 2773332.423],
 ['10967113574169', 308481.476, 2773329.017],
 ['10967113574169', 308475.489, 2773326.319],
 ['10967113574169', 308469.678, 2773324.052],
 ['10967113574169', 308453.346, 2773317.682],
 ['10967113574169', 308451.226, 2773316.857],
 ['10967113574169', 308446.581, 2773315.169],
 ['10967113574169', 308439.543, 2773312.984],
 ['10967113574169', 308436.822, 2773312.272],
 ['10967113574169', 308424.699, 2773309.131],
 ['10967113574169', 308421.539, 2773308.3],
 ['10967113574169', 308415.129, 2773306.616],
 ['10967113574169', 308409.505, 2773305.287],
 ['10967113574169', 308404.667, 2773304.15],
 ['10967113574169', 308393.057, 277330

In [123]:
coordinates_dict = {}

for item in result:
    # 取出每筆資料的編號（例如 '10967113574169'）
    identifier = item[0]
    # 取得對應的坐標資料 (x, y)
    coordinates = [item[1], item[2]]
    
    # 如果這個編號已經在字典裡，則將新的坐標資料加到現有的列表中
    if identifier not in coordinates_dict:
        coordinates_dict[identifier] = []  # 如果該編號還沒有資料，創建一個空列表
    coordinates_dict[identifier].append(coordinates)

display(coordinates_dict)

###############################################################################

# 定義 TWD97 和 WGS84 座標系統
twd97 = Proj(init='epsg:3826')  # TWD97
wgs84 = Proj(init='epsg:4326')  # WGS84

# 假設有多筆 TWD97 坐標 [[x1, y1], [x2, y2], ...]

conn = sqlite3.connect('TPEroad.db')
cursor = conn.cursor()


# 轉換所有 TWD97 坐標為 WGS84
coordinates_wgs84 = []
for key, value in coordinates_dict.items():
    coordinates_twd97 = value
    keyy = key
    for x, y in value:
        # 轉換為 WGS84 坐標
        longitude, latitude = transform(twd97, wgs84, x, y)
        coordinates_wgs84.append([longitude, latitude])
        # display(coordinates_wgs84)
        sql = '''
        INSERT INTO coordinates_test (BILL_CODE, lat, lon)
        SELECT ?, ROUND(?, 6), ROUND(?, 6)
        WHERE NOT EXISTS (
            SELECT 1 FROM coordinates_test WHERE lat = ROUND(?, 6)
        );
        '''
        cursor.execute(sql, (keyy, latitude, longitude, latitude))

conn.commit()
cursor.close()
conn.close()


{'10967113574169': [[308552.16, 2773353.307],
  [308546.447, 2773362.631],
  [308545.636, 2773362.174],
  [308539.956, 2773359.086],
  [308517.369, 2773346.862],
  [308510.022, 2773342.859],
  [308497.119, 2773336.425],
  [308489.019, 2773332.423],
  [308481.476, 2773329.017],
  [308475.489, 2773326.319],
  [308469.678, 2773324.052],
  [308453.346, 2773317.682],
  [308451.226, 2773316.857],
  [308446.581, 2773315.169],
  [308439.543, 2773312.984],
  [308436.822, 2773312.272],
  [308424.699, 2773309.131],
  [308421.539, 2773308.3],
  [308415.129, 2773306.616],
  [308409.505, 2773305.287],
  [308404.667, 2773304.15],
  [308393.057, 2773301.279],
  [308380.613, 2773298.3],
  [308390.234, 2773282.476],
  [308393.066, 2773283.682],
  [308405.821, 2773288.708],
  [308409.715, 2773290.256],
  [308426.728, 2773297.257],
  [308435.25, 2773299.903],
  [308440.96, 2773301.685],
  [308443.217, 2773302.485],
  [308465.978, 2773310.553],
  [308487.85, 2773319.679],
  [308503.874, 2773327.261],
  [30

c:\Users\ASUS\miniconda3\envs\venv1\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\ASUS\miniconda3\envs\venv1\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_23684\2176690072.py:35: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-py

In [ ]:

# 定義 TWD97 和 WGS84 座標系統
twd97 = Proj(init='epsg:3826')  # TWD97
wgs84 = Proj(init='epsg:4326')  # WGS84

# 假設有多筆 TWD97 坐標 [[x1, y1], [x2, y2], ...]
coordinates_twd97 = result[0]

# 轉換所有 TWD97 坐標為 WGS84
coordinates_wgs84 = []

for x, y in coordinates_twd97:
    # 轉換為 WGS84 坐標
    longitude, latitude = transform(twd97, wgs84, x, y)
    coordinates_wgs84.append([longitude, latitude])

# 輸出轉換後的所有 WGS84 坐標
for i, (lon, lat) in enumerate(coordinates_wgs84):
    print(f"Point {i+1}: Latitude={lon}, Longitude={lat}")


c:\Users\user\miniconda3\envs\venv\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\miniconda3\envs\venv\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\user\AppData\Local\Temp\ipykernel_14180\2136062567.py:13: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pypr

KeyboardInterrupt: 

In [ ]:
from pyproj import Transformer

# 創建轉換器
transformer = Transformer.from_crs("epsg:3826", "epsg:4326")
twd97_x = 305689.109
twd97_y = 2768207.525
# 轉換 TWD97 到 WGS84
latitude,longitude = transformer.transform(twd97_x, twd97_y)

print(f'TWD97轉WGS84: Latitude: {latitude:.7}, Longitude: {longitude:.7}')


TWD97轉WGS84: Latitude: 25.02091, Longitude: 121.5518


twd97是台灣的座標系統
WGS84是世界的座標系統
經度 longitude
緯度 latitude
所以台灣的X = longitude, Y = latitude

In [14]:
from geopy.geocoders import Nominatim

# 初始化 Nominatim 反向地理編碼器
geolocator = Nominatim(user_agent='PROJ')

def get_address_from_coordinates(lat, lon):
    """ 根據經緯度獲取地址或道路名稱 """
    location = geolocator.reverse((lat, lon), language='zh-TW', exactly_one=True)
    if location:
        return location.address
    return "未找到地址"

# 範例：查詢經緯度 (台北市)
latitude = 25.02066
longitude = 121.5518

address = get_address_from_coordinates(latitude, longitude)
print(address)
print(type(address))
print(f"座標 ({latitude}, {longitude}) 對應的地址是：{address}")


7號, 樂業街118巷, 芳和里, 大安區, 六張犁, 臺北市, 106, 臺灣
<class 'str'>
座標 (25.02066, 121.5518) 對應的地址是：7號, 樂業街118巷, 芳和里, 大安區, 六張犁, 臺北市, 106, 臺灣


In [47]:
address = "7號, 樂業街118巷, 芳和里, 大安區, 六張犁, 臺北市, 106, 臺灣"
list_address = [item.strip() for item in address.split(",")]
list_address.reverse()
print(list_address)
dic_address = {} 


['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街118巷', '7號']


In [23]:
display(type(response_json))
# print(response_json["features"][0]["properties"])
response_json01 = response_json["features"][0]["properties"]
for k,v in response_json01.items():
    print(f"{k}:{v}")

dict

BILL_CODE:10967113535908
URGENT_RCV_DATE:20241017
URGENT_START_DATE:20241116000000
URGENT_END_DATE:20241116220000
URGENT_ADDRESS1:台北市樂業街108巷1至11號
X1:305689.109
Y1:2768179.406
URGENT_ADDRESS2:台北市樂業街116號圍牆及118巷2~8號
X2:305689.109
Y2:2768179.406


In [3]:
    
def download_data():
    '''
    下載資料 需要做驗證只寫入不在資料庫裏面的那筆
    '''
    conn = sqlite3.connect("./TPEroad.db")
    url = 'https://tpnco.blob.core.windows.net/blobfs/Rally/TodayUrgentCase.json'
    try:
        response = requests.get(url)
        response.raise_for_status()
        with conn:
            cursor = conn.cursor()
            cursor.execute('''
            CREATE TABLE IF NOT EXISTS records (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                Bill_code TEXT,
                RCVdate TEXT,
                Start_date TEXT,
                End_date TEXT,
                Address1 TEXT,
                X1 TEXT,
                Y1 TEXT,
                Address2 TEXT,
                X2 TEXT,
                Y2 TEXT,
                UNIQUE(Address1,RCVdate)
            )
            ''')
            print("Table 'records' created or already exists.")
    except Exception as e:
        print(e)

    else:
        with conn:
            cursor = conn.cursor()
            for i in response_json['features']:
                for k in i['properties']:
                    Bill_code:str = i['properties']["BILL_CODE"]
                    RCVdate = i['properties']["URGENT_RCV_DATE"]
                    Start_date = i['properties']["URGENT_START_DATE"]
                    End_date = i['properties']["URGENT_END_DATE"]
                    Address1 = i['properties']["URGENT_ADDRESS1"]
                    X1 = i['properties']["X1"]
                    Y1 = i['properties']["Y1"]
                    Address2 = i['properties']["URGENT_ADDRESS2"]
                    X2 = i['properties']["X2"]
                    Y2 = i['properties']["Y2"]
                    
                    cursor = conn.cursor()
                    sql = '''INSERT OR IGNORE INTO records(Bill_code, RCVdate, Start_date, End_date, Address1, X1, Y1, Address2, X2, Y2)
                                values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
                            '''
                    cursor.execute(sql,(Bill_code, RCVdate, Start_date, End_date, Address1, X1, Y1, Address2, X2, Y2))
download_data()

Table 'records' created or already exists.


現在有每一個x,y轉換成的地址 再把地址存成我用的資料表 再從資料表裡面提取我需要的區的位置 就可以做combobox的下拉式選單

In [4]:
import functoins
import sqlite3

conn = sqlite3.connect('TPEroad.db')
with conn:
    cursor = conn.cursor()

    sql = '''
        SELECT Address1,x1,y1 FROM records
    '''

    cursor.execute(sql)
    # 把地址跟x1y1座標存進一個字典裡面
    address = [ ]
    for i in cursor.fetchall():
        di = {}
        di["地址"] = i[0]
        di["x1"] = i[1]
        di["y1"] = i[2]
        address.append(di)
    cursor.close() #把cursor關掉 清出記憶體
    display(address)

[{'地址': '北投區建民路１６１之５號', 'x1': '302384.879', 'y1': '2778195.38'},
 {'地址': '大安區敦化南路二段６３巷６４號', 'x1': '305668.124', 'y1': '2769314.433'},
 {'地址': '大安區臥龍街３１６號', 'x1': '306518.028', 'y1': '2767751.599'},
 {'地址': '松山區延壽街３１９－３２１號', 'x1': '306666.638', 'y1': '2772177.367'},
 {'地址': '大安區安居街２４號', 'x1': '306017.325', 'y1': '2768229.119'},
 {'地址': '松山區八德路三段１３７號', 'x1': '306001.614', 'y1': '2771219.82'},
 {'地址': '北投區明德路104號至108號', 'x1': '302435.414', 'y1': '2778037.181'},
 {'地址': '大安區大安路一段１７６巷５號', 'x1': '305016.716', 'y1': '2769932.84'},
 {'地址': '大安區和平東路一段87號  張立文理補習班', 'x1': '302991.933', 'y1': '2768872.108'},
 {'地址': '大安區溫州街５８巷14號', 'x1': '303683.128', 'y1': '2768161.802'},
 {'地址': '士林區士東路２６６巷３弄１２之１號', 'x1': '304193.216', 'y1': '2778349.522'},
 {'地址': '大安區永康街４２號', 'x1': '303447.801', 'y1': '2769138.498'},
 {'地址': '內湖區成功路四段６１巷３２弄４號', 'x1': '310035.908', 'y1': '2774866.6'},
 {'地址': '北投區自強街67-６９號', 'x1': '301893.092', 'y1': '2778128.963'},
 {'地址': '台北市文山區忠順街ㄧ段9巷1號', 'x1': '306441.723', 'y1': '2764127

接下來要提取每個字典裏面的x1y1然後丟到我的latlonturn裡面轉成地址(['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街', '114號'])  
再for i in list if '區' in i sql='insert ? into records' execute(sql,(i))


In [15]:
import functoins
import sqlite3
import time
conn = sqlite3.connect("TPEroad.db")
cursor = conn.cursor()
alladdress = []
for i in address:
    ad = i['地址']
    fuad = functoins.latlonturn("reverse",alladdress,i['x1'],i['y1'])
    fullstreet = ''.join(fuad)
    lat,lon = functoins.xytransform(i['x1'],i['y1'])
    for i in fuad:
        if "區" in i :dist = i
    add_content_sql = '''
    UPDATE RECORDS
    SET 新地址 = ?, 行政區 = ?, Lat = ?, Lon = ?
    WHERE address1 = ?;
    '''
    add_column_sql1='''
    ALTER TABLE RECORDS 
    ADD COLUMN 新地址 TEXT
    '''
    add_column_sql2='''
    ALTER TABLE RECORDS 
    ADD COLUMN 行政區 TEXT
    '''
    add_column_sql3='''
    ALTER TABLE RECORDS 
    ADD COLUMN Lat TEXT
    '''    
    add_column_sql4='''
    ALTER TABLE RECORDS 
    ADD COLUMN Lon TEXT
    '''

    add_column_sql5='''
    ALTER TABLE RECORDS 
    ADD COLUMN 日期 TEXT
    '''

    format_date='''
    UPDATE records 
	SET 日期=  (SELECT strftime('%Y-%m-%d', 
	              substr(RCVdate , 1, 4) || '-' || substr(RCVdate , 5, 2) || '-' || substr(RCVdate , 7, 2)) )
    '''


    cursor.execute(add_content_sql,(fullstreet, dist, lat, lon, ad))
    time.sleep(5)
conn.commit()


1.執行反轉操作
2.TWD97轉WGS84: Latitude: 25.1112, Longitude: 121.5194
3.單次轉換結果['臺灣', '112', '臺北市', '蘭雅', '北投區', '裕民里', '建民路', '161之11號']
4.總清單:[['臺灣', '112', '臺北市', '蘭雅', '北投區', '裕民里', '建民路', '161之11號']]
2.TWD97轉WGS84: Latitude: 25.1112, Longitude: 121.5194
1.執行反轉操作
2.TWD97轉WGS84: Latitude: 25.03091, Longitude: 121.5516
3.單次轉換結果['臺灣', '106', '臺北市', '車罾', '大安區', '義安里', '敦化南路二段63巷', '56號']
4.總清單:[['臺灣', '112', '臺北市', '蘭雅', '北投區', '裕民里', '建民路', '161之11號'], ['臺灣', '106', '臺北市', '車罾', '大安區', '義安里', '敦化南路二段63巷', '56號']]
2.TWD97轉WGS84: Latitude: 25.03091, Longitude: 121.5516
1.執行反轉操作
2.TWD97轉WGS84: Latitude: 25.01677, Longitude: 121.56
3.單次轉換結果['臺灣', '10676', '臺北市', '六張犁', '大安區', '黎和里', '臥龍街']
4.總清單:[['臺灣', '112', '臺北市', '蘭雅', '北投區', '裕民里', '建民路', '161之11號'], ['臺灣', '106', '臺北市', '車罾', '大安區', '義安里', '敦化南路二段63巷', '56號'], ['臺灣', '10676', '臺北市', '六張犁', '大安區', '黎和里', '臥龍街']]
2.TWD97轉WGS84: Latitude: 25.01677, Longitude: 121.56
1.執行反轉操作
2.TWD97轉WGS84: Latitude: 25.05672, Longitude: 121.5616
3.單次轉換結果['臺灣'

In [14]:
import functoins
address = [{'地址': '台北市樂業街108巷1至11號', 'x1': '305689.109', 'y1': '2768179.406'},
 {'地址': '士林區大東路１７６號', 'x1': '302989.777', 'y1': '2776097.833'}]
alladdress = []
# for i in address:
#     ad = i['地址']
#     fuad = functoins.latlonturn("reverse",alladdress,i['x1'],i['y1'])
#     type(fuad)
# for i in fuad:
#     if "區" in i :dist = i
#     print(dist)

fuad = functoins.latlonturn('reverse', alladdress, address[0]["x1"], address[0]["y1"])
fullstreet = ''.join(fuad)
display(fullstreet)

1.執行反轉操作
2.TWD97轉WGS84: Latitude: 25.02066, Longitude: 121.5518
3.單次轉換結果['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街118巷', '7號']
4.總清單:[['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街118巷', '7號']]


'臺灣106臺北市六張犁大安區芳和里樂業街118巷7號'

In [4]:
import sqlite3
def get_district()->list[str]:
    '''
    docString
    parameter:
    return:
        傳出所有的行政區名稱
    '''
    conn = sqlite3.connect("TPEroad.db")
    with conn:
        # Create a cursor object to execute SQL commands
        cursor = conn.cursor()
        # SQL query to select unique sitenames from records table
        sql = '''
        SELECT DISTINCT 行政區
        FROM records
        '''
        # Execute the SQL query
        cursor.execute(sql)
        # Get all results and extract first item from each row into a list
        counties = [items[0] for items in cursor.fetchall()]
    
    # Return the list of unique sitenames
    return counties
get_district()

['北投區',
 '大安區',
 '松山區',
 '士林區',
 '內湖區',
 '文山區',
 '南港區',
 '中山區',
 '大同區',
 '萬華區',
 '中正區',
 '信義區']

tkmap做兩點距離或範圍

In [ ]:
import tkinter as tk
from tkinter import ttk
import tkintermapview as tkmap
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

# 創建 Tkinter 視窗
root = tk.Tk()
root.title("./tkintermapview Example")
map_frame = ttk.Frame()
map_widget = tkmap.TkinterMapView(map_frame,width=400,height=400,corner_radius=0)
map_widget.set_position(25.02091606,121.55139558,marker=True)
map_widget.pack()
map_frame.pack(padx=10,pady=10)

#定義要顯示的地點（經度、緯度）
locations = [
    {"name": "Point A", "lat": 25.02091606, "lon": 121.55139558},  # San Francisco
    {"name": "Point B", "lat": 25.02091585, "lon": 121.55139533},  # Los Angeles
    {"name": "Point C", "lat": 25.02075568, "lon": 121.55121300}   # New York
]

# 在地圖上標註這些點
for location in locations:
    map_widget.set_marker(location["lat"], location["lon"], text=location["name"])


# 啟動 Tkinter 主循環
root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import ttk
import tkintermapview

# 創建 Tkinter 主窗口
root = tk.Tk()
root.title("./tkintermapview Example")

# 設置地圖視窗
map_frame = ttk.Frame(root)
map_widget = tkintermapview.TkinterMapView(map_frame, width=600, height=400, corner_radius=0)

# 設定初始地圖顯示位置（台北市）
map_widget.set_position(25.02091606, 121.55139558, marker=True)

# 三個點的經緯度
points = [
    {"name": "Point 1", "lat": 25.0330, "lon": 121.5654},  # 例如台北101
    {"name": "Point 2", "lat": 25.0340, "lon": 121.5644},  # 點2
    {"name": "Point 3", "lat": 25.0320, "lon": 121.5664},  # 點3
]

# 在地圖上添加這三個點的標記
for point in points:
    map_widget.set_marker(point["lat"], point["lon"], text=point["name"])

# 計算這三個點的範圍（bounding box）
lats = [point["lat"] for point in points]
lons = [point["lon"] for point in points]
min_lat, max_lat = min(lats), max(lats)
min_lon, max_lon = min(lons), max(lons)

# 計算範圍的中心點和放大等級
center_lat = (min_lat + max_lat) / 2
center_lon = (min_lon + max_lon) / 2

# 設置地圖顯示範圍
map_widget.set_position(center_lat, center_lon, zoom=15)

# 畫出範圍的矩形
map_widget.add_polygon(min_lat, min_lon, max_lat, max_lon, color="blue", width=2)

# 打包顯示地圖
map_widget.pack()
map_frame.pack(padx=10, pady=10)

# 啟動 Tkinter 主循環
root.mainloop()


AttributeError: 'TkinterMapView' object has no attribute 'add_polygon'

: 